<a href="https://colab.research.google.com/github/rato42/linear_regression/blob/unstable-yeah-sure/CLASSBASED_new_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports


In [1]:
import sys

# Detect Colab environment
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    from google.colab import drive, auth
    from googleapiclient.discovery import build
    from googleapiclient.http import MediaIoBaseDownload
    !pip install pwlf

    import pwlf

    auth.authenticate_user()
    drive.mount("/content/drive")
else:
    import pwlf
    from oauth2client.service_account import ServiceAccountCredentials

    print("Running outside Colab (VS Code/local)")

try:
    import gspread
except ModuleNotFoundError:
    if IN_COLAB and "google.colab" in str(get_ipython()):
        %pip install gspread
    import gspread

from gspread_dataframe import set_with_dataframe

Running outside Colab (VS Code/local)


In [2]:

import os

import time

import math
import scipy.stats as stats
import pandas as pd
import numpy as np

from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.families import Poisson
from statsmodels.genmod.cov_struct import Autoregressive
from sklearn.linear_model import Ridge
import statsmodels.api as sm

from openpyxl import load_workbook
from openpyxl.styles import Alignment

from scipy.stats import linregress, ttest_ind, ttest_rel, ttest_1samp
import matplotlib.pyplot as plt
import re
from openpyxl.drawing.image import Image
from matplotlib.font_manager import FontProperties
from io import BytesIO
import statsmodels.formula.api as smf
import statsmodels.api as sm

import numpy as np
import plotly.graph_objects as go


from datetime import datetime, timedelta

from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tools.tools import add_constant
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer

from dateutil.relativedelta import relativedelta

from sklearn import set_config
set_config(display="diagram")

import logging
from typing import Dict, Any

import io

# LOAD --- data from gsheet if colab or local


In [3]:
dados_brutos = None
output_wksheet = None

input_path = "https://docs.google.com/spreadsheets/d/1m3eEaxWT4Unb8jBZWKjiLivavfA0x3PT1F1Rz1eXwVE/edit?gid=0"
output_path = "https://docs.google.com/spreadsheets/d/1RzC3DfKNUwYA-qfUN0i53hUwRkWGMBOCoYQqdqG4MQY/edit?gid=0"

local_input_path = (
    #"D:\\CodeStuff\\Stats\\colab_linear_regression\\linear_regression\\input\\input.csv" ### poa
    #"D:\\CodeStuff\\Stats\\colab_linear_regression\\linear_regression\\input\\input_backup_com_aps_total_RS.xlsx" ### rs
    #"D:\\CodeStuff\\Stats\\colab_linear_regression\\linear_regression\\input\\input_backup_com_aps_total.xlsx"
    #"D:\\CodeStuff\\Stats\\colab_linear_regression\\linear_regression\\input\\input_backup_com_aps_total_RS_percentual.xlsx"
    #"D:\\CodeStuff\\Stats\\colab_linear_regression\\linear_regression\\input\\input_IR_1000_poa.xlsx"
    #"D:\\CodeStuff\\Stats\\colab_linear_regression\\linear_regression\\input\\input_IR_1000_brasil.xlsx"
    "D:\\CodeStuff\\Stats\\colab_linear_regression\\linear_regression\\input\\regions_IR_1000.xlsx"
    #"D:\\CodeStuff\\Stats\\colab_linear_regression\\linear_regression\\input\\raw_regions_joined.xlsx"
)

output_wksheet = None

if IN_COLAB:
    import gspread
    from google.auth import default

    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    workbook = gc.open_by_url(input_path)
    worksheets = workbook.worksheets()
    output_wksheet = gc.open_by_url(output_path)
    dados_brutos = worksheets[0].get_all_values()
else:
    try:
        dados_brutos = pd.read_csv(local_input_path, header=None).values.tolist()
    except:
        dados_brutos = pd.read_excel(local_input_path, header=None).values.tolist()

# TODO LIST

PROMPT
im doing a ITS. here is the structure
data is monthly, from 2016 to 2024
A PraisWinsten regression

the following dummies:
Pandemia_Step [2020 - 2022] - binary
Pandemia_Trend [2020 - 2022] - increase by 1 by period
PosPandemia_Step [2022 - 2024] - binary
PosPandemia_Trend [2022 - 2024] - increase by 1 by period
Lockdown 2020 [03/2020 - 05/2020] - pulse, binary
Locdown 2021 [04/2021 - 05/2021] - pulse, binary

In [4]:
#Parte 1 – Análise geral com grupo controle
#
#    ITS com controle (como já falamos)
#
#    Mostra que saúde mental cresceu mais que os atendimentos gerais
#
#    Mostra os efeitos da pandemia nesse comportamento
#
#Parte 2 – Análise dos subgrupos (sem controle)
#
#    ITS separado para cada diagnóstico (ex: CID-10 F32, F41, etc.)
#
#    Sem grupo controle, mas comparando antes/depois da pandemia
#
#    Pode usar o mesmo modelo (Prais-Winsten com step + trend)

In [5]:
#Omnibus Test: The high value (41.163) and the p-value (< 0.0001) 
# suggest that the residuals deviate from normality, which might indicate some model specification issues.
#
#Jarque-Bera Test: Similarly, this test suggests non-normality in the residuals.

In [6]:
### to test if step is necessary

'''
import statsmodels.api as sm

# model with step
mod_full = sm.OLS(y, X_full).fit()

# model without step
mod_reduced = sm.OLS(y, X_reduced).fit()

# compare using F-test
f_test_result = mod_full.compare_f_test(mod_reduced)
'''

'\nimport statsmodels.api as sm\n\n# model with step\nmod_full = sm.OLS(y, X_full).fit()\n\n# model without step\nmod_reduced = sm.OLS(y, X_reduced).fit()\n\n# compare using F-test\nf_test_result = mod_full.compare_f_test(mod_reduced)\n'

In [7]:
# TODO: Adicionar dummy de interação para comparar SM e outros atendimentos

# TODO: Orthogonalise pospand trend (so to not have colinearity with months)

# TODO: analisar overfitting

# TODO: taxa. a população se mantem estavel? os serviços de saude se mantem estaveis?

# https://chatgpt.com/canvas/shared/68084b682c548191a92db280344e5474

--- Low HIGH COMPLEXITY

- [x] Data Quality & Exploration  
  - [x] Plot the raw time series and annotate intervention points.  
  - [x] Check for missing values, outliers, or data entry errors.  
  - [x] Verify your date index is consistent (e.g., monthly frequency).  

- [x] Descriptive Statistics  
  - [x] Compute summary statistics (mean, variance) before and after the pandemic.  
  - [x] Calculate and plot moving averages to smooth trends.  

- [x] Dummy Variable Audit  
  - [x] Confirm each dummy is coded 0/1 for the correct periods.  
  - [x] Review naming consistency and placement in the DataFrame.  

- [x] Basic Model Fit  
  - [x] Fit the current segmented regression (consults ~ time + level_dummy + slope_dummy + month_dummies + lockdown_outliers).  
  - [x] Inspect R², coefficient signs, and p-values to sanity-check effects.  Export this to a readeable format

--- MEDIUM COMPLEXITY

- [ ] Residual Diagnostics  
  - [X] Plot residuals vs. fitted values to check heteroscedasticity.  
  - [ ] Examine ACF/PACF of residuals:  
    - [X] First using OLS (to confirm the need for AR structure).  
    - [X] Then using Prais-Winsten (to assess remaining autocorrelation).  

- [ ] Autocorrelation & Robust SE  
  - [ ] If autocorrelation exists, adjust standard errors (e.g., Newey-West) or fit an autoregressive error structure (e.g., AR(1)).  
  - [ ] Since you’re using Prais-Winsten, this step may be partially addressed — verify with residuals.  

- [ ] Seasonality & Trend Flexibility  
  - [ ] Test alternative seasonal structures (e.g., Fourier terms) instead of month dummies.  
  - [ ] Consider adding a quadratic or spline-based time trend.  

- [ ] Outlier Handling  
  - [ ] Justify and potentially consolidate lockdown outlier dummies (pulse vs. step).  
  - [ ] Compare models with and without these outlier dummies (use AIC/BIC).  

- [ ] Multicollinearity Check  
  - [ ] Compute Variance Inflation Factors (VIFs) for covariates.  

--- HIGH COMPLEXITY

- [ ] Adicionar dummy de interação para comparar SM e outros atendimentos

- [ ] Time-Series Cross-Validation  
  - [ ] Implement rolling-origin evaluation to assess predictive performance.  
  - [ ] Compare cross-validated errors (e.g., RMSE) across model specs to detect overfitting.  

- [ ] Model Comparison & Selection  
  - [ ] Use information criteria (AIC/BIC) to compare alternative models.  
  - [ ] Evaluate nested models (e.g., with/without slope change) via likelihood ratio tests.  

- [ ] Advanced Interrupted Time Series Methods  
  - [ ] Explore Bayesian ITS or segmented regression with autoregressive integrated moving average (ARIMA) errors.  
  - [x] Consider structural break tests (e.g., Chow test) for formal change-point detection.  

- [ ] Sensitivity & Robustness  
  - [ ] Run sensitivity analyses using different intervention start/end dates.  
  - [ ] Test robustness to alternative outlier definitions or lag structures.  

- [ ] Forecasting & Counterfactuals  
  - [ ] Generate counterfactual predictions for the post-pandemic period.  
  - [ ] Visualize observed vs. counterfactual to illustrate intervention impact.

In [8]:
def build_model_formula(dependant_var, independant_var=False, dummy_names=[]):
    """
    Builds the regression formula dynamically.

    Parameters:
    -----------
    dependant_var : str
        The dependent variable (Y).
    independant_var : str or bool
        The independent variable (X). If False, it will be excluded.
    dummy_names : list
        List of dummy variable names to include in the formula.

    Returns:
    --------
    str
        The regression formula.
    """
    base_model_formula = f"{dependant_var} ~ "
    if independant_var:
        base_model_formula += f"{independant_var} + "

    # Add dummy variables (including Fourier terms if present)
    model_formula = base_model_formula
    if dummy_names:
        for i, dummy in enumerate(dummy_names):
            model_formula += dummy
            if i != len(dummy_names) - 1:  # Add '+' if not the last element
                model_formula += " + "

    return model_formula.strip(" + "), base_model_formula.strip(" + ")


def build_dummy_names(dummies_args, include_month_dummies, include_quarter_dummies, include_fourier_terms=False, n_harmonics=1):
    """
    Builds the list of dummy variable names dynamically.

    Parameters:
    -----------
    dummies_args : dict
        Dictionary of dummy variable configurations.
    include_month_dummies : bool
        Whether to include month dummies.
    include_quarter_dummies : bool
        Whether to include quarter dummies.
    include_fourier_terms : bool
        Whether to include Fourier terms for seasonality.
    n_harmonics : int
        Number of Fourier harmonics to include.

    Returns:
    --------
    list
        List of dummy variable names.
    """
    dummy_names = []

    # Add dummy variables from dummies_args
    for dummy_name in dummies_args.keys():
        dummy_names.append(dummy_name)

    # Add Fourier terms if enabled
    if include_fourier_terms:
        for k in range(1, n_harmonics + 1):
            dummy_names.append(f"cos{k}")
            dummy_names.append(f"sin{k}")

    # Add month dummies if enabled
    if include_month_dummies:
        for i in range(1, 12):
            dummy_names.append("month_" + str(i + 1))

    # Add quarter dummies if enabled
    if include_quarter_dummies:
        for i in range(1, 4):
            dummy_names.append("quarter_" + str(i + 1))

    return dummy_names


# ARGS --- Define Args


In [9]:
### ARGS


RAW_ATD_VAR = 'valores'
RAW_TOTAL_ATD_VAR = 'total_atd'
IR_VAR = 'IR'
IR_SCALE = 1000
DATE_NAME = "date"
GROUP_COL_NAME = "variavel"
LOCAL_OUTPUT_PATH = "D:\\CodeStuff\\Stats\\colab_linear_regression\\linear_regression\\output\\output.xlsx"


MAIN_SERIE = '_Grand Total'
CONTROL_SERIE = '__Atendimentos na APS (exceto saúde mental)'

MOVING_WINDOW = 12
PLOT_MEDIA_MOVEL = False


### model vars
INDEPENDENT_VAR = "periodo"  ### Y
DEPENDENT_VAR = RAW_ATD_VAR #"valores"  ### X


######### DUMMIES ##########

MEDIA_MOVEL_AS_FEATURE = False
MEDIA_MOVEL_AS_DEPENDENT = False

INCLUDE_FOURIER_TERMS = True
N_HARMONICS = 1
FOURIER_PERIOD = 12
INCLUDE_MONTH_DUMMIES = False
INCLUDE_QUARTER_DUMMIES = False

PRE_START_DATE = "2016-01-01"  ### data inicial para o modelo, se for menor que a data do dataframe, o modelo vai dar erro

### as quedas do primeiro lockdown nao são tão claras e a recuperação e queda parecem ser mais graduais
CHOQUE_INICIAL_START_DATE= "2020-03-01"
CHOQUE_INICIAL_END_DATE = "2020-07-01"#"2020-08-01" #month based

### abril e maio tiveram quedas bem identificaveis nos atendimentos em todas variavieis
SECONDLOCKDOWN_START_DATE = "2021-04-01"
SECONDLOCKDOWN_END_DATE = '2021-05-01' #month based  #'2021-04-01'#quarter_based #

PANDEMIA_START_DATE = "2020-04-01"
PANDEMIA_END_DATE = "2022-03-01" #"2021-02-01" 




PANDEMIA2_START_DATE = "2021-03-01"
PANDEMIA2_END_DATE = "2022-03-01" 

POS_PANDEMIA_START_DATE = "2022-04-01"
END_DATE = "2024-12-01"#"2024-12-01"  ### data final para o modelo, se for maior que a data do dataframe, o modelo vai dar erro




OMNICRON_START_DATE = "2022-01-01"
OMNICRON_END_DATE = "2022-02-01"  ### data final para o modelo, se for maior que a data do dataframe, o modelo vai dar erro

'''
DUMMY_ARGS =        {
        'Choque_Inicial': {"start_date": CHOQUE_INICIAL_START_DATE, "end_date": CHOQUE_INICIAL_END_DATE},
        'Lockdown_2021': {"start_date": SECONDLOCKDOWN_START_DATE, "end_date": SECONDLOCKDOWN_END_DATE},
        #'Enchentes_2024': {"start_date": "2024-05-01", "end_date": "2024-05-01"},
        'Step_PosPandemia': {"start_date": POS_PANDEMIA_START_DATE, "end_date": END_DATE,'exclude_ranges': POS_PANDEMIA_EXCLUDE_RANGES, 'plot':False, 'step': True,},
        'Trend_PosPandemia' : {"start_date": POS_PANDEMIA_START_DATE, "end_date": END_DATE,'exclude_ranges': POS_PANDEMIA_EXCLUDE_RANGES, 'plot': True, 'add_time_trend': True, 'keep_period_index': False, 'step': False},
        }
'''

POS_PANDEMIA_EXCLUDE_RANGES = [
    #(CHOQUE_INICIAL_START_DATE, CHOQUE_INICIAL_END_DATE),
    #(SECONDLOCKDOWN_START_DATE, SECONDLOCKDOWN_END_DATE),
    #("2024-05-01", "2024-05-01"),
                               ]

PANDEMIA_EXCLUDE_RANGES = [
    #(CHOQUE_INICIAL_START_DATE, CHOQUE_INICIAL_END_DATE),
    #(SECONDLOCKDOWN_START_DATE, SECONDLOCKDOWN_END_DATE),
    #(OMNICRON_START_DATE, OMNICRON_END_DATE),
]

TESTE_REMOVE_OUTLIER_START = "2019-10-01"
TESTE_REMOVE_OUTLIER_END = "2019-12-01"  ### data final para o modelo, se for maior que a data do dataframe, o modelo vai dar erro
PREPANDEMIA_EXCLUDE_RANGES = [
    #(TESTE_REMOVE_OUTLIER_START, TESTE_REMOVE_OUTLIER_END),
]

TREND_KEEP_PERIOD_INDEX = False  ### se True, o modelo vai manter o index do periodo, se False, o modelo vai usar o index do dataframe original (sem os dummies)

DUMMY_ARGS =        {
        
        
        #'PrePandemia_Step': {"start_date": PRE_START_DATE, "end_date": PANDEMIA_START_DATE, 'exclude_ranges': PANDEMIA_EXCLUDE_RANGES,},
        #'PrePandemia_Trend' : {"start_date": PRE_START_DATE, "end_date": PANDEMIA_START_DATE,'exclude_ranges': PREPANDEMIA_EXCLUDE_RANGES, 'add_time_trend': True, 'keep_period_index': False},
        
        'Pandemia_Step': {"start_date": PANDEMIA_START_DATE, "end_date": PANDEMIA_END_DATE, 'exclude_ranges': PANDEMIA_EXCLUDE_RANGES,},
        'Pandemia_Trend' : {"start_date": PANDEMIA_START_DATE, "end_date": PANDEMIA_END_DATE,'exclude_ranges': PANDEMIA_EXCLUDE_RANGES, 'add_time_trend': True, 'keep_period_index': TREND_KEEP_PERIOD_INDEX},
        
        #'Pandemia2_Step': {"start_date": PANDEMIA2_START_DATE, "end_date": PANDEMIA2_END_DATE, 'exclude_ranges': PANDEMIA_EXCLUDE_RANGES,},
        #'Pandemia2_Trend' : {"start_date": PANDEMIA2_START_DATE, "end_date": PANDEMIA2_END_DATE,'exclude_ranges': PANDEMIA_EXCLUDE_RANGES, 'add_time_trend': True, 'keep_period_index': TREND_KEEP_PERIOD_INDEX},

        'PosPandemia_Step': {"start_date": POS_PANDEMIA_START_DATE, "end_date": END_DATE,'exclude_ranges': POS_PANDEMIA_EXCLUDE_RANGES, 'plot':False, 'step': True,},
        'PosPandemia_Trend' : {"start_date": POS_PANDEMIA_START_DATE, "end_date": END_DATE,'exclude_ranges': POS_PANDEMIA_EXCLUDE_RANGES, 'plot': True, 'add_time_trend': True, 'keep_period_index': TREND_KEEP_PERIOD_INDEX, 'step': False},
        
        ### Pulse Outliers
        #'teste_remove_outlier': {"start_date": TESTE_REMOVE_OUTLIER_START, "end_date": TESTE_REMOVE_OUTLIER_END,},
        #'Lockdown_2020_Step': {"start_date": CHOQUE_INICIAL_START_DATE, "end_date": CHOQUE_INICIAL_END_DATE},
        #'Lockdown_2021_Step': {"start_date": SECONDLOCKDOWN_START_DATE, "end_date": SECONDLOCKDOWN_END_DATE},
        #'Omnicron_2022': {"start_date": OMNICRON_START_DATE, "end_date": OMNICRON_END_DATE},
        #'Enchentes_2024': {"start_date": "2024-05-01", "end_date": "2024-05-01"},
        }



##############################


PREPROCESSOR_ARGS = {
    'DateRange': [2016, 2024],
    'FrequencyDateGroup' : 'M',
    'DummyArgs' : DUMMY_ARGS,
    'melt_total_atd': True,
    'ExtractIR': True,
}



###### PROCESSING ######


DUMMY_NAMES = build_dummy_names(
          DUMMY_ARGS,
          INCLUDE_MONTH_DUMMIES,
          INCLUDE_QUARTER_DUMMIES,
          include_fourier_terms=INCLUDE_FOURIER_TERMS,
          n_harmonics=N_HARMONICS,
          )

MODEL_FORMULA, BASE_MODEL_FORMULA = build_model_formula(DEPENDENT_VAR,
                                                        INDEPENDENT_VAR,
                                                        dummy_names = DUMMY_NAMES,

                                                        )






PROCESSOR_ARGS = {
    "formula": MODEL_FORMULA,#"valores ~ periodo",  # Example formula
    "model":  smf.glsar,#smf.ols,# # Use OLS as the regression model for testing
    "find_best_rho": True,#True,  # Disable best rho search for simplicity
    "rho": None,
    "cov_type": "HAC",  # Use nonrobust covariance for testing
    "cov_kwargs": {'maxlags': 4},  # No additional covariance arguments
    "iterative_fit": True,  # Enable iterative fitting if needed
    "max_iterations": 50,  # Set maximum iterations for iterative fitting
    "find_best_rho_args": {
        "start": -1.00,
        "stop": 1.01,
        "step": 0.01,
        "criterio": "aic"
    },
}


GEE_PROCESSOR_ARGS = {
    'formula': MODEL_FORMULA,
    "family": Poisson(),  # Use Poisson family for count data
    "cov_struct": Autoregressive(),  # Use autoregressive covariance structure
    "groups": GROUP_COL_NAME,
}



c:\Users\Lucas\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\genmod\cov_struct.py:796: FutureWarning: grid=True will become default in a future version
  warnings.warn(


In [10]:
MODEL_FORMULA, BASE_MODEL_FORMULA


('valores ~ periodo + Pandemia_Step + Pandemia_Trend + PosPandemia_Step + PosPandemia_Trend + cos1 + sin1',
 'valores ~ periodo')

# Define Preprocessors


In [11]:
def BuildDataDF(data):
    string_dates = data[0][1:]  # Store original string dates

    # Converter as strings de datas em objetos datetime
    #date_list = [datetime.strptime(date, "%d/%m/%Y") for date in string_dates]
    date_list = [
    date if isinstance(date, datetime) else datetime.strptime(date, "%d/%m/%Y")
    for date in string_dates
    ]

    # Extrair os nomes das variáveis
    variavel = [row[0] for row in data[1:]]

    # Extrair os dados (sem agrupamento)
    dados = [row[1:] for row in data[1:]]
    dados = np.array(dados, dtype=float)  # Converter para tipo object para evitar warning

    # Criar o DataFrame with DatetimeIndex
    df = pd.DataFrame(data=dados, index=variavel, columns=pd.DatetimeIndex(date_list))

    return df

class DateFilterTransformer(BaseEstimator, TransformerMixin):
    def __init__(
        self, periodos_inclusivos=None, periodos_exclusivos=None, date_col=DATE_NAME
    ):
        self.periodos_inclusivos = periodos_inclusivos
        self.periodos_exclusivos = periodos_exclusivos
        self.date_col = date_col

    def get_date_range(self, Y1, Y2=None, M1=None, M2=None, D1=None, D2=None):
        Y2 = Y2 or Y1
        M1 = M1 or 1
        M2 = M2 or 12
        D1 = D1 or 1
        D2 = D2 or 31
        start_date = datetime(Y1, M1, D1)
        end_date = datetime(Y2, M2, D2)
        return pd.date_range(start=start_date, end=end_date, freq="D")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # Convert date column if it's not datetime yet
        if not pd.api.types.is_datetime64_any_dtype(X[self.date_col]):
            X[self.date_col] = pd.to_datetime(X[self.date_col], errors="coerce")

        if self.periodos_inclusivos:
            valid_dates = self.get_date_range(*self.periodos_inclusivos)
            return X[X[self.date_col].isin(valid_dates)].reset_index(drop=True)

        elif self.periodos_exclusivos:
            invalid_dates = self.get_date_range(*self.periodos_exclusivos)
            return X[~X[self.date_col].isin(invalid_dates)].reset_index(drop=True)

        return X

class MeltTransformer(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        var_name=GROUP_COL_NAME,
        value_name=RAW_ATD_VAR,
        date_name=DATE_NAME,
        y_name=INDEPENDENT_VAR,
        perform_IR=False,
        total_suffix="_total",
        total_column_name=RAW_TOTAL_ATD_VAR,
    ):
        self.var_name = var_name
        self.value_name = value_name
        self.date_name = date_name
        self.y_name = y_name
        self.perform_IR = perform_IR
        self.total_suffix = total_suffix
        self.total_column_name = total_column_name

    def fit(self, X, y=None):
        # Nothing to learn, so just return self
        return self

    def transform(self, X):
        # Separate rows with the `_total` suffix
        if self.perform_IR:
            reserved_rows = X[X.index.str.endswith(self.total_suffix)]
            remaining_rows = X[~X.index.str.endswith(self.total_suffix)]

            # Transpose and reset index for the remaining rows
            df_long = remaining_rows.T.reset_index()

            # Rename index to date_name
            df_long = df_long.rename(columns={"index": self.date_name})

            # Melt the DataFrame
            df_long = pd.melt(
                df_long,
                id_vars=[self.date_name],
                var_name=self.var_name,
                value_name=self.value_name,
            )

            # Sort by category and date
            df_final = df_long.sort_values([self.var_name, self.date_name]).reset_index(
                drop=True
            )

            # Add the `total_atd` column
            reserved_rows = reserved_rows.T.reset_index().rename(columns={"index": self.date_name})
            reserved_rows_long = pd.melt(
                reserved_rows,
                id_vars=[self.date_name],
                var_name=self.var_name,
                value_name=self.total_column_name,
            )

            # Remove the `_total` suffix from the var_name in reserved_rows_long
            reserved_rows_long[self.var_name] = reserved_rows_long[self.var_name].str.replace(
                self.total_suffix, "", regex=False
            )

            # Merge the `total_atd` column into the final DataFrame
            df_final = df_final.merge(
                reserved_rows_long,
                on=[self.date_name, self.var_name],
                how="left",
            )
        else:
                        # Transpose and reset index for the remaining rows
            df_long = X.T.reset_index()

            # Rename index to date_name
            df_long = df_long.rename(columns={"index": self.date_name})

            # Melt the DataFrame
            df_long = pd.melt(
                df_long,
                id_vars=[self.date_name],
                var_name=self.var_name,
                value_name=self.value_name,
            )

            # Sort by category and date
            df_final = df_long.sort_values([self.var_name, self.date_name]).reset_index(
                drop=True
            )


        return df_final

class PeriodIndexAdder(BaseEstimator, TransformerMixin):
    def __init__(self, group_col=GROUP_COL_NAME, new_col=INDEPENDENT_VAR):
        self.group_col = group_col
        self.new_col = new_col
        self.period_date_mapping = {}  # Dictionary to store period-date equivalences

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X[self.new_col] = X.groupby(self.group_col).cumcount() + 1

        #Store the mapping of periods to dates
        self.period_date_mapping = dict(zip(X[DATE_NAME], X[self.new_col]))
        #for group, group_data in X.groupby(self.group_col):
        #    self.period_date_mapping[group] = dict(zip(group_data[DATE_NAME], group_data[self.new_col]))

        return X

class DataFrameSorter(BaseEstimator, TransformerMixin):
    """
    Sorts a DataFrame by a specified column.

    Parameters:
    -----------
    sort_by : str
        Name of the column to sort by.
    ascending : bool, default=True
        Whether to sort in ascending order.
    """

    def __init__(self, sort_by, ascending=True):
        self.sort_by = sort_by
        self.ascending = ascending

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        if self.sort_by not in X.columns:
            raise ValueError(f"Column '{self.sort_by}' not found in DataFrame.")
        return X.sort_values(by=self.sort_by, ascending=self.ascending).reset_index(drop=True)

class MonthlyDummyAdder(BaseEstimator, TransformerMixin):
    """
    Adds dummy variables for each month to control for seasonality, leaving December as the baseline.

    Parameters:
    -----------
    date_column : str
        Name of the column containing dates (must be datetime).
    """

    def __init__(self, date_column=DATE_NAME):
        self.date_column = date_column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X[self.date_column] = pd.to_datetime(X[self.date_column])  # Ensure dates are in datetime format

        # Extract month from the date column
        X['month'] = X[self.date_column].dt.month

        # Create dummy variables for each month, excluding December as the baseline
        month_dummies = pd.get_dummies(X['month'], prefix="month", drop_first=True).astype(int)



        # Concatenate the dummy variables with the original DataFrame
        X = pd.concat([X, month_dummies], axis=1)

        # Drop the temporary 'month' column
        X.drop(columns=['month'], inplace=True)

        return X

class QuarterlyDummyAdder(BaseEstimator, TransformerMixin):
    """
    Adds dummy variables for each quarter to control for seasonality, leaving Q4 as the baseline.

    Parameters:
    -----------
    date_column : str
        Name of the column containing dates (must be datetime).
    """

    def __init__(self, date_column=DATE_NAME):
        self.date_column = date_column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X[self.date_column] = pd.to_datetime(X[self.date_column])  # Ensure dates are in datetime format

        # Extract quarter from the date column
        X['quarter'] = X[self.date_column].dt.quarter

        # Create dummy variables for each quarter, excluding Q4 as the baseline
        quarter_dummies = pd.get_dummies(X['quarter'], prefix="quarter", drop_first=True).astype(int)

        # Concatenate the dummy variables with the original DataFrame
        X = pd.concat([X, quarter_dummies], axis=1)

        # Drop the temporary 'quarter' column
        X.drop(columns=['quarter'], inplace=True)

        return X

class MultiDummyAdder(BaseEstimator, TransformerMixin):
    """
    Adds multiple dummy variables based on date ranges or custom conditions.
    Allows each dummy to be binary or a local time trend via its own config.

    Parameters:
    -----------
    dummy_config : dict
        Dictionary defining dummy variables. Example:
        {
            'Pandemia': {
                'start_date': '2020-01-01',
                'end_date': '2020-12-31',
                'add_time_trend': True,
                'exclude_ranges': [
                    ('2020-04-01', '2020-05-01'),
                    ('2020-07-01', '2020-08-01')
                ]  # Optional
            }
        }
    date_column : str
        Name of the column containing dates (must be datetime).
    """

    def __init__(self, dummy_config, date_column=DATE_NAME):
        self.dummy_config = dummy_config
        self.date_column = date_column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X[self.date_column] = pd.to_datetime(X[self.date_column])
        X = X.sort_values(self.date_column).reset_index(drop=True)

        for dummy_name, config in self.dummy_config.items():
            start_date = pd.to_datetime(config.get("start_date"))
            end_date = pd.to_datetime(config.get("end_date"))
            use_trend = config.get("add_time_trend", False)
            keep_index = config.get("keep_period_index", False)
            exclude_ranges = config.get("exclude_ranges", [])  # List of ranges to exclude

            if not start_date:
                raise ValueError(f"Missing 'start_date' for dummy '{dummy_name}'.")

            # Create the initial mask for the date range
            if end_date is not None:
                mask = (X[self.date_column] >= start_date) & (X[self.date_column] <= end_date)
            else:
                mask = X[self.date_column] >= start_date

            # Exclude multiple ranges within the period
            for excl_start, excl_end in exclude_ranges:
                excl_start = pd.to_datetime(excl_start)
                excl_end = pd.to_datetime(excl_end)
                mask &= ~((X[self.date_column] >= excl_start) & (X[self.date_column] <= excl_end))

            # Apply trend or binary dummy logic
            if use_trend:
                if keep_index:
                    # Global period multiplied by the dummy
                    X[dummy_name] = 0
                    X.loc[mask, dummy_name] = X.loc[mask, INDEPENDENT_VAR]
                else:
                    start_periodo = X.loc[mask, INDEPENDENT_VAR].min()
                    X[dummy_name] = 0
                    X.loc[mask, dummy_name] = (X.loc[mask, INDEPENDENT_VAR] - start_periodo + 1).clip(lower=0)
            else:
                X[dummy_name] = mask.astype(int)

        return X

class FrequencyGroupingTransformer(BaseEstimator, TransformerMixin):
    """
    ## only datetime supported groupings
    Transformer for grouping data by standard datetime frequencies.

    This transformer takes a DataFrame with datetime columns and groups
    the data by a specified frequency (e.g., 'Q' for quarterly, 'M' for monthly).

    ARGS:
        frequency (str): The frequency to group by. Defaults to 'Q' (quarterly).
                        Supported frequencies are those accepted by pd.Grouper (e.g., 'Q', 'M', 'Y').
    """

    def __init__(self, frequency="Q"):  # Default to quarterly grouping
        self.frequency = frequency

    def fit(self, X, y=None):
        """
        This transformer does not require fitting.

        ARGS:
            X (pd.DataFrame): The input DataFrame.
            y (Any, optional): Ignored. Defaults to None.

        Returns:
            self: Returns the transformer instance.
        """
        return self

    def transform(self, X):
        """
        Groups the data by the specified frequency and sums the values.

        ARGS:
            X (pd.DataFrame): The input DataFrame with datetime columns.

        Returns:
            pd.DataFrame: The grouped DataFrame with summed values.
                          Columns are datetime objects representing the end of each group.
        """
        if self.frequency == None or self.frequency == 'M':
            return X
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame.")



        # Convert columns to datetime if they are not already
        if not isinstance(X.columns[0], pd.Timestamp):
            X.columns = pd.to_datetime(X.columns, errors="coerce")

        # Group by the specified frequency using pd.Grouper
        grouped_df = X.T.groupby(pd.Grouper(freq=self.frequency)).sum().T

        return grouped_df







In [12]:
class MovingAverageTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, group_col=GROUP_COL_NAME, date_col=DATE_NAME, value_col=DEPENDENT_VAR, window=3, prefix='ma', center=False):
        """
        window: tamanho da janela
        prefix: prefixo para o nome da nova coluna
        center: se True, a janela será centralizada
        """
        self.window = window
        self.prefix = prefix
        self.center = center
        self.value_col = value_col
        self.group_col = group_col
        self.date_col = date_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        def rolling_mean_by_group(group):
            group = group.sort_values(by=self.date_col)  # Sort by date within group
            new_col = f"{self.prefix}_{self.window}_{self.value_col}"
            group[new_col] = group[self.value_col].rolling(window=self.window, center=self.center, min_periods=1).mean()
            return group

        # Apply the rolling mean and reset the index to preserve the original structure
        X = X.groupby(self.group_col, group_keys=False).apply(rolling_mean_by_group)

        return X

In [13]:
class FourierSeasonalityAdder(BaseEstimator, TransformerMixin):
    """
    Adds sine and cosine Fourier terms for seasonality based on the month of the date.

    Parameters:
    -----------
    date_column : str
        Name of the column containing dates (must be datetime).
    n_harmonics : int
        Number of Fourier harmonics to add (default is 1).
    """

    def __init__(self, date_column=DATE_NAME, n_harmonics=1):
        self.date_column = date_column
        self.n_harmonics = n_harmonics

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X[self.date_column] = pd.to_datetime(X[self.date_column])  # Ensure dates are in datetime format

        # Extract time (month number starting from 1)
        X['month'] = X[self.date_column].dt.month

        # Add Fourier terms
        for k in range(1, self.n_harmonics + 1):
            X[f'cos{k}'] = np.cos(2 * np.pi * k * (X['month'] - 1) / 12)  # Month to 0-based
            X[f'sin{k}'] = np.sin(2 * np.pi * k * (X['month'] - 1) / 12)  # Month to 0-based

        #X.drop(columns=['month'], inplace=True)

        return X

In [14]:
from statsmodels.tsa.deterministic import Fourier

class FourierSeasonalityAdder(BaseEstimator, TransformerMixin):
    def __init__(self, date_column=DATE_NAME, period=12, order=1):
        """
        Parameters:
        -----------
        date_column : str
            Name of the column containing dates (must be datetime).
        period : int
            The period of the seasonality (e.g., 12 for monthly data).
        order : int
            The number of Fourier harmonics to include.
        """
        self.date_column = date_column
        self.period = period
        self.order = order
        self.fourier = Fourier(period=self.period, order=self.order)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # Ensure the date column is in datetime format
        X[self.date_column] = pd.to_datetime(X[self.date_column])

        # Ensure unique index
        if not X.index.is_unique:
            X = X.reset_index(drop=True)

        # Generate Fourier terms
        fourier_terms = self.fourier.in_sample(X[self.date_column])

        # Rename Fourier columns to use sin1, cos1, etc.
        renamed_columns = {col: f"sin{int(i/2)+1}" if i % 2 else f"cos{int(i/2)+1}" 
                           for i, col in enumerate(fourier_terms.columns)}
        fourier_terms.rename(columns=renamed_columns, inplace=True)

        # Concatenate Fourier terms with the original DataFrame
        X = pd.concat([X.reset_index(drop=True), fourier_terms.reset_index(drop=True)], axis=1)

        # Debugging: Check the range of Fourier terms
        for column in fourier_terms.columns:
            min_value = fourier_terms[column].min()
            max_value = fourier_terms[column].max()
            if min_value < -1 or max_value > 1:
                print(f"Warning: {column} is out of range [-1, 1]. Min: {min_value}, Max: {max_value}")
            else:
                print(f"{column} is within the correct range [-1, 1]. Min: {min_value}, Max: {max_value}")

        return X

# EXEC --- Instantiate Pipe and get RegressionDF

In [15]:

pipe = Pipeline(
    [
        ("grouping", FrequencyGroupingTransformer(frequency=PREPROCESSOR_ARGS['FrequencyDateGroup'])),
        ("melt", MeltTransformer(perform_IR=PREPROCESSOR_ARGS['melt_total_atd'])),
        ("date_filter", DateFilterTransformer(periodos_inclusivos=PREPROCESSOR_ARGS['DateRange'])),
        ("period_index_adder", PeriodIndexAdder()),
        ("dummy_adder", MultiDummyAdder(dummy_config=PREPROCESSOR_ARGS['DummyArgs'])),
        #("monthly_dummy_adder", MonthlyDummyAdder()),
        #("quarterly_dummy_adder", QuarterlyDummyAdder()),
        ('fourier_seasonality', FourierSeasonalityAdder(period = FOURIER_PERIOD,order=N_HARMONICS)),
        #('moving_average', MovingAverageTransformer(window=MOVING_WINDOW, prefix='ma', center=True)),
        ("sorter", DataFrameSorter(sort_by=GROUP_COL_NAME, ascending=False)),


    ]
)


RawDataDF = BuildDataDF(dados_brutos)
RegressionDF = pipe.fit_transform(RawDataDF)

if PREPROCESSOR_ARGS['ExtractIR']:
    RegressionDF[IR_VAR] =  RegressionDF[RAW_ATD_VAR]/RegressionDF[RAW_TOTAL_ATD_VAR] * IR_SCALE

if RegressionDF[RAW_TOTAL_ATD_VAR] is not None:
    RegressionDF["offset"] = np.log(RegressionDF["total_atd"] / IR_SCALE)


RegressionDF


cos1 is within the correct range [-1, 1]. Min: -1.0, Max: 1.0
sin1 is within the correct range [-1, 1]. Min: -1.0, Max: 1.0


,date,variavel,valores,total_atd,periodo,Pandemia_Step,Pandemia_Trend,PosPandemia_Step,PosPandemia_Trend,cos1,sin1,IR,offset
0,2024-12-01,sul,30.584808,NaN,108,0,0,1,33,-5.000000e-01,8.660254e-01,NaN,NaN
1,2023-05-01,sul,27.334337,NaN,89,0,0,1,14,5.000000e-01,8.660254e-01,NaN,NaN
2,2018-04-01,sul,24.709625,NaN,28,0,0,0,0,-1.000000e+00,1.961292e-15,NaN,NaN
3,2023-03-01,sul,29.578098,NaN,87,0,0,1,12,1.000000e+00,2.302769e-14,NaN,NaN
4,2018-05-01,sul,23.912230,NaN,29,0,0,0,0,8.660254e-01,5.000000e-01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,2022-04-01,brasil,25.702799,NaN,76,0,0,1,1,-8.660254e-01,-5.000000e-01,NaN,NaN
644,2017-11-01,brasil,16.826766,NaN,23,0,0,0,0,5.000000e-01,-8.660254e-01,NaN,NaN
645,2019-08-01,brasil,21.377674,NaN,44,0,0,0,0,-5.000000e-01,-8.660254e-01,NaN,NaN
646,2017-12-01,brasil,17.619795,NaN,24,0,0,0,0,6.369401e-15,-1.000000e+00,NaN,NaN


# Exploração Pré intervenção


In [16]:
def run_pre_intervention_diagnostics(data, date_col, dependent_var, intervention_start_date):
    """
    Realiza testes diagnósticos no período pré-intervenção.

    Args:
        data (pd.DataFrame): DataFrame contendo os dados.
        date_col (str): Nome da coluna de datas.
        dependent_var (str): Nome da variável dependente.
        intervention_start_date (str): Data de início da intervenção (formato 'YYYY-MM-DD').

    Returns:
        dict: Resultados dos testes diagnósticos.
    """
    import pandas as pd
    import matplotlib.pyplot as plt
    from statsmodels.tsa.stattools import adfuller
    from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
    from statsmodels.stats.diagnostic import het_breuschpagan
    from statsmodels.stats.stattools import durbin_watson
    from scipy.stats import shapiro, jarque_bera, normaltest
    import statsmodels.api as sm

    # Filtrar o período pré-intervenção
    pre_intervention_df = data[data[date_col] < pd.to_datetime(intervention_start_date)]

    # Inicializar resultados
    results = {}

    # 1. Teste de Estacionariedade (ADF)
    y = pre_intervention_df[dependent_var]
    adf_result = adfuller(y)
    results['ADF Statistic'] = adf_result[0]
    results['ADF p-value'] = adf_result[1]
    results['ADF Stationary'] = adf_result[1] < 0.05

    # 2. Diagnóstico de Autocorrelação (ACF e PACF)
    results['ACF'] = "Plotted"
    results['PACF'] = "Plotted"

    # 3. Teste de Heterocedasticidade (Breusch-Pagan)
    X = sm.add_constant(pre_intervention_df.index)  # Índice como variável independente
    model = sm.OLS(y, X).fit()
    bp_test = het_breuschpagan(model.resid, X)
    results['Breusch-Pagan LM Statistic'] = bp_test[0]
    results['Breusch-Pagan p-value'] = bp_test[1]
    results['Heteroscedasticity'] = bp_test[1] < 0.05

    # 4. Testes de Normalidade dos Resíduos
    shapiro_test = shapiro(model.resid)
    jb_test = jarque_bera(model.resid)
    dagostino_test = normaltest(model.resid)
    results['Shapiro-Wilk p-value'] = shapiro_test.pvalue
    results['Jarque-Bera p-value'] = jb_test[1]
    results['D\'Agostino p-value'] = dagostino_test.pvalue
    results['Normality'] = all(p > 0.05 for p in [shapiro_test.pvalue, jb_test[1], dagostino_test.pvalue])

    # 5. Teste de Autocorrelação (Durbin-Watson)
    dw_stat = durbin_watson(model.resid)
    results['Durbin-Watson Statistic'] = dw_stat
    results['No Autocorrelation'] = 1.5 < dw_stat < 2.5

    return results


# Consolidar os resultados em um DataFrame
diagnostics_results = []

for variable in RegressionDF[GROUP_COL_NAME].unique():
    print(f"Analisando variável: {variable}")
    variable_df = RegressionDF[RegressionDF[GROUP_COL_NAME] == variable].copy()
    diagnostics = run_pre_intervention_diagnostics(
        data=variable_df,
        date_col=DATE_NAME,
        dependent_var=DEPENDENT_VAR,
        intervention_start_date=PANDEMIA_START_DATE
    )
    diagnostics['Variable'] = variable  # Adicionar o nome da variável ao resultado
    diagnostics_results.append(diagnostics)

# Criar um DataFrame com os resultados
diagnostics_df = pd.DataFrame(diagnostics_results)
diagnostics_df.set_index('Variable', inplace=True)

# Exibir o DataFrame
diagnostics_df


Analisando variável: sul
Analisando variável: sudeste
Analisando variável: norte
Analisando variável: nordeste
Analisando variável: centro_oeste
Analisando variável: brasil


,ADF Statistic,ADF p-value,ADF Stationary,ACF,PACF,Breusch-Pagan LM Statistic,Breusch-Pagan p-value,Heteroscedasticity,Shapiro-Wilk p-value,Jarque-Bera p-value,D'Agostino p-value,Normality,Durbin-Watson Statistic,No Autocorrelation
Variable,,,,,,,,,,,,,,
sul,-4.176008,7.233005e-04,True,Plotted,Plotted,4.833701,0.027909,True,0.154287,0.192087,1.296530e-01,True,1.098450,False
sudeste,-3.882759,2.165456e-03,True,Plotted,Plotted,6.498025,0.010799,True,0.028896,0.507291,4.488718e-01,False,2.066701,True
norte,-5.657226,9.559254e-07,True,Plotted,Plotted,10.482236,0.001205,True,0.000859,0.095624,3.633776e-07,False,1.936615,True
nordeste,-7.561956,2.993784e-11,True,Plotted,Plotted,0.194999,0.658788,False,0.018868,0.280674,1.735519e-01,False,2.207011,True
centro_oeste,-1.581328,4.929659e-01,False,Plotted,Plotted,11.534394,0.000683,True,0.000631,0.093825,3.725673e-09,False,0.313522,False
brasil,-5.936927,2.309143e-07,True,Plotted,Plotted,2.498358,0.113965,False,0.248070,0.683315,7.133238e-01,True,2.157164,True


# Media movel 

In [17]:
import matplotlib.pyplot as plt



moving_avg_pipe = Pipeline([
    ('moving_average', MovingAverageTransformer(window=MOVING_WINDOW, prefix='ma', center=True)),
    ]
    )

def plot_moving_average(df, column, ma_column=None, title='Moving Average Plot', date_column='date'):
    """
    Plota a série temporal original junto com a média móvel.

    df: DataFrame contendo os dados
    column: nome da coluna original
    ma_column: nome da coluna da média móvel (se None, assume padrão de 'ma_{window}_{column}')
    title: título do gráfico
    date_column: nome da coluna de data
    """
    if ma_column is None:
        ma_column = [col for col in df.columns if col.startswith('ma_') and col.endswith(column)]
        if ma_column:
            ma_column = ma_column[0]
        else:
            raise ValueError("Média móvel não encontrada. Verifique o nome da coluna.")

    df[date_column] = pd.to_datetime(df[date_column])  # Ensure datetime format

    plt.figure(figsize=(12, 5))
    plt.plot(df[date_column], df[column], label='Original', color='lightgray')
    plt.plot(df[date_column], df[ma_column], label=f'{ma_column}', color='blue')
    plt.title(title)
    plt.xlabel('Data')
    plt.ylabel(column)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

if PLOT_MEDIA_MOVEL:
    for var in RegressionDF[GROUP_COL_NAME].unique():
        var_df = RegressionDF[RegressionDF[GROUP_COL_NAME] == var]
        #var_df = var_df.sort_values(by = INDEPENDENT_VAR, ascending=False)
        #print(var_df)
        #var_df = moving_avg_pipe.fit_transform(var_df)


        plot_moving_average(var_df, column=DEPENDENT_VAR, ma_column=f'ma_{MOVING_WINDOW}_valores', title=f'Moving Average Plot for {var}')



# Get Period Date Map and Dummies Period Indexes

In [18]:
PERIOD_DATE_MAP = pipe.named_steps['period_index_adder'].period_date_mapping

In [19]:
def find_idx_from_date(target_date, args, period_date_map=PERIOD_DATE_MAP):
    """
    Finds the 'periodo' (index) corresponding to a given datetime in the PERIOD_DATE_MAP.

    Args:
        target_date (str or datetime): The target date to search for (e.g., '2020-01-01').
        period_date_map (dict): The PERIOD_DATE_MAP containing period-to-date mappings.

    Returns:
        int: The 'periodo' (index) corresponding to the target date, or None if not found.
    """
    # Ensure the target_date is a datetime object and normalize to quarter-end
    if isinstance(target_date, str):
        target_date = pd.to_datetime(target_date)
    if args['FrequencyDateGroup'] == 'Q' or args['FrequencyDateGroup'] == 'QE':
        target_date = target_date + pd.offsets.QuarterEnd(0)

    # Iterate through the PERIOD_DATE_MAP to find the matching period
    for date, period in period_date_map.items():
        if isinstance(date, str):
            date = pd.to_datetime(date)
        if args['FrequencyDateGroup'] == 'Q' or args['FrequencyDateGroup'] == 'QE':
            date = date + pd.offsets.QuarterEnd(0) # Normalize to quarter-end
        if date == target_date:
            return period - 1

    return None


def find_date_from_idx(target_idx, args, period_date_map=PERIOD_DATE_MAP):
    """
    Finds the date corresponding to a given 'periodo' (index) in the PERIOD_DATE_MAP.

    Args:
        target_idx (int): The target index to search for (e.g., 1, 2, 3).
        period_date_map (dict): The PERIOD_DATE_MAP containing period-to-date mappings.

    Returns:
        datetime or None: The date corresponding to the target index, or None if not found.
    """
    # Iterate through the PERIOD_DATE_MAP to find the matching index
    for date, period in period_date_map.items():
        if period == target_idx:
            date = pd.to_datetime(date) if isinstance(date, str) else date
            date = date + pd.offsets.QuarterEnd(0) if args['FrequencyDateGroup'] == 'Q' or args['FrequencyDateGroup'] == 'QE' else date
            return date
    return None

In [20]:
DummyIndexes = {}

# Iterate over DUMMY_ARGS to calculate and store indexes
for dummy_name, config in DUMMY_ARGS.items():
    start_date = config.get("start_date")
    end_date = config.get("end_date")

    print(dummy_name, config)
    # Calculate indexes using find_idx_from_date
    start_idx = find_idx_from_date(start_date, PREPROCESSOR_ARGS, PERIOD_DATE_MAP) if start_date else None
    end_idx = find_idx_from_date(end_date, PREPROCESSOR_ARGS, PERIOD_DATE_MAP) if end_date else None

    # Store the indexes in the table
    DummyIndexes[dummy_name] = {
        "start_index": start_idx,
        "end_index": end_idx
    }
DummyIndexes

Pandemia_Step {'start_date': '2020-04-01', 'end_date': '2022-03-01', 'exclude_ranges': []}
Pandemia_Trend {'start_date': '2020-04-01', 'end_date': '2022-03-01', 'exclude_ranges': [], 'add_time_trend': True, 'keep_period_index': False}
PosPandemia_Step {'start_date': '2022-04-01', 'end_date': '2024-12-01', 'exclude_ranges': [], 'plot': False, 'step': True}
PosPandemia_Trend {'start_date': '2022-04-01', 'end_date': '2024-12-01', 'exclude_ranges': [], 'plot': True, 'add_time_trend': True, 'keep_period_index': False, 'step': False}


{'Pandemia_Step': {'start_index': 51, 'end_index': 74},
 'Pandemia_Trend': {'start_index': 51, 'end_index': 74},
 'PosPandemia_Step': {'start_index': 75, 'end_index': 107},
 'PosPandemia_Trend': {'start_index': 75, 'end_index': 107}}

# Exploration


In [21]:
from statsmodels.api import OLS
from statsmodels.tools.tools import add_constant
import pandas as pd
from scipy.stats import f
main_regrdf = RegressionDF.copy()
def chow_test(data, split_point, dependent_var, independent_vars):
    """
    Realiza o teste de Chow para verificar mudanças estruturais em um modelo de regressão.

    Args:
        data (pd.DataFrame): DataFrame contendo os dados.
        split_point (int): Ponto de divisão para o teste de Chow.
        dependent_var (str): Nome da variável dependente.
        independent_vars (list): Lista de nomes das variáveis independentes.

    Returns:
        dict: Resultados do teste de Chow contendo os valores F e p.
    """
    # Dividir os dados em dois subconjuntos
    data1 = data[data['periodo'] <= split_point]
    data2 = data[data['periodo'] > split_point]

    # Ajustar os modelos para cada subconjunto
    X1 = add_constant(data1[independent_vars])
    y1 = data1[dependent_var]
    model1 = OLS(y1, X1).fit()

    X2 = add_constant(data2[independent_vars])
    y2 = data2[dependent_var]
    model2 = OLS(y2, X2).fit()

    # Ajustar o modelo para o conjunto completo
    X_full = add_constant(data[independent_vars])
    y_full = data[dependent_var]
    model_full = OLS(y_full, X_full).fit()

    # Calcular os graus de liberdade
    k = len(independent_vars) + 1  # Número de parâmetros (incluindo o intercepto)
    n1 = len(data1)
    n2 = len(data2)
    n = n1 + n2

    # Calcular a soma dos quadrados dos resíduos
    RSS1 = model1.ssr
    RSS2 = model2.ssr
    RSS_full = model_full.ssr

    # Calcular o valor F
    F = ((RSS_full - (RSS1 + RSS2)) / k) / ((RSS1 + RSS2) / (n - 2 * k))

    # Calcular o p-valor
    p_value = 1 - f.cdf(F, k, n - 2 * k)

    return {"F-statistic": F, "p-value": p_value}


if False:
    chow_results = {}
    split_point = find_idx_from_date(POS_PANDEMIA_START_DATE, PREPROCESSOR_ARGS)  # Exemplo: ponto de divisão para o teste de Chow
    dependent_var = DEPENDENT_VAR
    independent_vars = [INDEPENDENT_VAR] #+ [key for key in DUMMY_ARGS.keys()]
    for variable in main_regrdf[GROUP_COL_NAME].unique():
        variable_data = main_regrdf[main_regrdf[GROUP_COL_NAME] == variable]
        result = chow_test(variable_data, split_point, dependent_var, independent_vars)
        chow_results[variable] = result
    # Converter os resultados em um DataFrame para visualização
    chow_results_df = pd.DataFrame(chow_results).T
    chow_results_df.columns = ["F-statistic", "p-value"]
    chow_results_df['p-value'] = chow_results_df['p-value'].round(8)
    print(chow_results_df)

In [22]:
### TODO: make the knots per variable
### TODO: remove the bumps/step_dummies before testing

pwmodel = pwlf.PiecewiseLinFit(main_regrdf[INDEPENDENT_VAR].values,main_regrdf[DEPENDENT_VAR].values)
knots = pwmodel.fit(2)
print(knots)
for knot in knots:
    print(find_date_from_idx(np.round(knot), PREPROCESSOR_ARGS, PERIOD_DATE_MAP))

[  1.          51.99999995 108.        ]
2016-01-01 00:00:00
2020-04-01 00:00:00
2024-12-01 00:00:00


In [23]:
import pandas as pd
import numpy as np
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_ljungbox
from statsmodels.stats.stattools import durbin_watson
from scipy.stats import shapiro, normaltest, jarque_bera
import statsmodels.api as sm

def analyze_time_series(df: pd.DataFrame, variable_col: str, date_col: str, value_col: str):
    """
    Analyzes a melted DataFrame for heteroscedasticity, distribution, and autocorrelation.

    Parameters:
        df (pd.DataFrame): Melted DataFrame with columns for variable, date, and value.
        variable_col (str): Column name for the variable identifier.
        date_col (str): Column name for the date.
        value_col (str): Column name for the value.

    Returns:
        dict: A dictionary with analysis results for each variable.
    """
    results = {}

    for variable in df[variable_col].unique():
        # Filter data for the current variable
        variable_data = df[df[variable_col] == variable].sort_values(by=date_col)
        y = variable_data[value_col].values
        x = np.arange(len(y))  # Time index as independent variable

        # Add constant for regression
        X = sm.add_constant(x)

        # Fit OLS model
        model = sm.OLS(y, X).fit()

        # Heteroscedasticity tests
        bp_test = het_breuschpagan(model.resid, X)
        bp_pvalue = bp_test[1]

        # Normality tests
        shapiro_test = shapiro(model.resid)
        shapiro_pvalue = shapiro_test.pvalue

        jb_test = jarque_bera(model.resid)
        jb_pvalue = jb_test[1]

        dagostino_test = normaltest(model.resid)
        dagostino_pvalue = dagostino_test.pvalue

        # Autocorrelation tests
        dw_stat = durbin_watson(model.resid)
        ljungbox_test = acorr_ljungbox(model.resid, lags=[10], return_df=True)
        ljungbox_pvalue = ljungbox_test['lb_pvalue'].iloc[0]

        # Store results
        results[variable] = {
            "Heteroscedasticity (Breusch-Pagan)": bp_pvalue,
            "Normality (Shapiro-Wilk)": shapiro_pvalue,
            "Normality (Jarque-Bera)": jb_pvalue,
            "Normality (D'Agostino)": dagostino_pvalue,
            "Autocorrelation (Durbin-Watson)": dw_stat,
            "Autocorrelation (Ljung-Box)": ljungbox_pvalue,
        }

    return results

explorationresults = analyze_time_series(main_regrdf, GROUP_COL_NAME, DATE_NAME, DEPENDENT_VAR)
explorationresultsDF = pd.DataFrame(explorationresults).T
explorationresultsDF

,Heteroscedasticity (Breusch-Pagan),Normality (Shapiro-Wilk),Normality (Jarque-Bera),Normality (D'Agostino),Autocorrelation (Durbin-Watson),Autocorrelation (Ljung-Box)
sul,0.164576,1.838493e-05,3.832472e-17,0.000152,0.662502,1.694655e-19
sudeste,0.750972,1.132293e-02,9.731833e-05,0.003396,0.473111,1.677197e-41
norte,0.000316,3.030665e-01,1.520846e-01,0.175068,0.406020,5.485827e-37
nordeste,0.224652,5.338393e-01,8.921544e-01,0.782065,0.591856,2.022385e-28
centro_oeste,0.616810,1.629422e-01,6.499785e-01,0.505295,0.446845,3.788040e-40
brasil,0.379582,7.713189e-07,2.743380e-33,0.000001,0.660199,7.826211e-17


# Define Processors

In [24]:
class RegressionProcessor:
    def __init__(
        self,
        data,
        formula,
        model=None,
        find_best_rho=False,
        find_best_rho_args={
            'start': -1.0,
            "stop": 1.0,
            'step': 0.05,
            'criterio': 'aic',
        },
        rho=None,
        cov_type=None,
        cov_kwargs=None,
        iterative_fit=False,  # New argument to enable iterative fitting
        max_iterations=100,  # Maximum iterations for iterative fitting
    ):
        self.data = data
        self.formula = formula
        self.model = model
        self.rho = rho
        self.cov_type = cov_type
        self.cov_kwargs = cov_kwargs
        self.find_best_rho = find_best_rho
        self.find_best_rho_args = find_best_rho_args
        self.iterative_fit = iterative_fit
        self.max_iterations = max_iterations
        self.results = None

    def fit(self, rho=None):
        # Fit the regression model using the specified formula and data
        if self.find_best_rho:
            best_rho = 1.0
            best_criterion_value = np.inf
            model = None
            results = None
            for rho in np.arange(
                self.find_best_rho_args['start'],
                self.find_best_rho_args['stop'],
                self.find_best_rho_args['step']
            ):
                test_model = self.model(self.formula, data=self.data, rho=rho)
                if self.iterative_fit and hasattr(test_model, "iterative_fit"):
                    test_results = test_model.iterative_fit(
                        maxiter=self.max_iterations,
                        cov_type=self.cov_type,
                        cov_kwds=self.cov_kwargs,
                    )
                else:
                    test_results = test_model.fit(
                        cov_type=self.cov_type,
                        cov_kwds=self.cov_kwargs,
                    )
                criterion_value = getattr(test_results, self.find_best_rho_args['criterio'])
                if criterion_value < best_criterion_value:
                    best_criterion_value = criterion_value
                    best_rho = rho
                    model = test_model
                    results = test_results
            self.rho = best_rho
            self.results = results
        else:
            if self.rho is not None:
                model = self.model(self.formula, data=self.data, rho=rho)
            else:
                model = self.model(self.formula, data=self.data)

            # Use iterative fitting if enabled and supported
            if self.iterative_fit and hasattr(model, "iterative_fit"):
                self.results = model.iterative_fit(
                    maxiter=self.max_iterations,
                    cov_type=self.cov_type,
                    cov_kwds=self.cov_kwargs,
                )
            else:
                self.results = model.fit(
                    cov_type=self.cov_type,
                    cov_kwds=self.cov_kwargs,
                )

In [25]:
class VariableProcessor:
    def __init__(
        self,
        df,
        processor_args,
        processor,
        evaluate = False,
        test_size=0.2,
        random_state=42,

    ):
        self.df = df
        self.results = {}
        self.processor_args = processor_args
        self.processor = processor
        self.evaluate = evaluate
        self.evaluation_results = {}
        self.test_size = test_size
        self.random_state = random_state


    def process_variables(self):
        # Process each variable in the DataFrame
        for variable in self.df[GROUP_COL_NAME].unique():
            variable_data = self.df[self.df[GROUP_COL_NAME] == variable].sort_values(by=DATE_NAME)
            variable_data = variable_data.reset_index(drop=True)

            processor = self.processor(data=variable_data, **self.processor_args)
            processor.fit()
            
            self.results[variable] = processor.results




            # Split into train and test
            if self.evaluate:
                train_data, test_data = train_test_split(
                    variable_data, test_size=self.test_size, random_state=self.random_state, shuffle=False
                )

                # Store results for train and test
                train_predictions = processor.results.predict(train_data)
                test_predictions = processor.results.predict(test_data)

                # Calculate metrics
                train_r2 = r2_score(train_data[DEPENDENT_VAR], train_predictions)
                test_r2 = r2_score(test_data[DEPENDENT_VAR], test_predictions)
                #n_train = len(train_data)
                #n_test = len(test_data)
                #p = len(processor.results.params) - 1
                train_mse = mean_squared_error(train_data[DEPENDENT_VAR], train_predictions)
                test_mse = mean_squared_error(test_data[DEPENDENT_VAR], test_predictions)

                evaluation_value = train_r2 - test_r2
                #evaluation_value_adj = train_r2_adj - test_r2_adj


                self.evaluation_results[variable] = {
                    #'Absolute Diff': absolute_difference,
                    'TrainMSE - TestMSE': train_mse - test_mse,
                    "Evaluation Value (r_train² - r_test²)": evaluation_value,
                    "Overfitting Risk": "High" if evaluation_value > 0.1 else "Low",
                    "Train R2": train_r2,
                    "Test R2": test_r2,
                    #"Evaluation Value (r_train_adj² - r_test_adj²)": evaluation_value_adj,
                    #"Overfitting Risk (adj_r2)": "High" if evaluation_value_adj > 0.1 else "Low",
                    #"Adj Train R2": train_r2_adj,
                    #"Adj Test R2": test_r2_adj,
                    "Train MSE": train_mse,
                    "Test MSE": test_mse,
                    "Processor Results": processor.results,
                    'splitted_data': {'train_data': train_data, 'test_data': test_data},
                }

        self.evaluation_results = pd.DataFrame(self.evaluation_results).T

        return self.results, self.evaluation_results

# Define Exporter

In [27]:
def export_results(results, output_path, processor_args, preprocessor_args, output_gsheet = None):
    """
    Exports the regression results and metadata to an Excel file.

    Args:
        results (dict): Dictionary containing regression results.
        output_path (str): Path to the output Excel file.
        processor_args (dict): Dictionary containing processor arguments.
        preprocessor_args (dict): Dictionary containing preprocessor arguments.
    """
    def extract_durbin_watson_from_summary(result):
        summary_text = result.summary().as_text()
        for line in summary_text.split('\n'):
            if "Durbin-Watson" in line:
                return float(line.split()[-1])  # Extract the last value in the line
        return None  # Return None if not found


    def try_create_worksheet(output_gsheet, title: str) -> None:
        if output_gsheet:
            try:
                output_gsheet.worksheet(title)
            except gspread.exceptions.WorksheetNotFound:
                output_gsheet.add_worksheet(title=title, rows="100", cols="20")


    stats_data = []
    coefficients_data = []

    for variable, result in results.items():
        # Extract coefficients
        coefficients = result.params
        coef_row = {"Variable": variable}
        for coef_name, coef_value in coefficients.items():
            coef_row[f"{coef_name}__Value"] = round(coef_value, 7)
            coef_row[f"{coef_name}__Value_%_intercept"] = coef_value/result.params['Intercept']*100
            coef_row[f"{coef_name}__P-Value"] = round(result.pvalues.get(coef_name, np.nan), 7)
            coef_row[f"{coef_name}__Std Err"] = round(result.bse.get(coef_name, np.nan), 7)
            coef_row[f"{coef_name}__Significant"] = result.pvalues.get(coef_name, np.nan) < 0.05  # At 5% significance
        coefficients_data.append(coef_row)

        # Extract other statistics
        stats_data.append({
            "Variable": variable,
            "R-Squared": round(result.rsquared, 7),
            "Adjusted R-Squared": round(result.rsquared_adj, 7),
            "Durbin-Watson": round(extract_durbin_watson_from_summary(result), 7),
            "AIC": round(result.aic, 7),
            "BIC": round(result.bic, 7),
            "Number of Observations": result.nobs,
            "F-Value": round(result.fvalue, 7) if result.fvalue is not None else np.nan,
            "F P-Value": round(result.f_pvalue, 7) if result.f_pvalue is not None else np.nan,
            "Log-Likelihood": round(result.llf, 7),
            "Condition Number": round(result.condition_number, 7) if result.condition_number is not None else np.nan
        })

    # Convert to DataFrames
    coefficients_df = pd.DataFrame(coefficients_data).set_index("Variable")
    stats_df = pd.DataFrame(stats_data).set_index("Variable")

    # Prepare metadata
    metadata = {
        "Processor Args": [str(processor_args)],
        "Preprocessor Args": [str(preprocessor_args)]
    }
    metadata_df = pd.DataFrame(metadata)

    # Write to gsheet
    if output_gsheet is not None:
        for title, clean_df, include_index in [
            ("Coefficients", coefficients_df, True),
            ("Statistics", stats_df, True),
            ("Metadata", metadata_df, False)
        ]:
            try_create_worksheet(title=title, output_gsheet=output_gsheet)
            ws = output_gsheet.worksheet(title)
            ws.clear()
            set_with_dataframe(ws, clean_df, include_index=include_index)
    # Write to Excel
    else:
       file_exists = os.path.exists(output_path)

               # Attempt to open file and catch PermissionError if the file is open elsewhere
       if file_exists:
           while True:
               try:
                   with pd.ExcelWriter(output_path) as writer:
                       # Write coefficients to one sheet
                       coefficients_df.to_excel(writer, sheet_name="Coefficients")

                       # Write other stats to another sheet
                       stats_df.to_excel(writer, sheet_name="Statistics")

                       # Write metadata to another sheet
                       metadata_df.to_excel(writer, sheet_name="Metadata", index=False)
                   # Load the workbook to apply formatting
                   workbook = load_workbook(output_path)
                   if "Residuals" not in workbook.sheetnames:
                       workbook.create_sheet("Residuals")
                       residuals_sheet = workbook["Residuals"]

                   # Plot residuals for each variable and add them to the Residuals sheet
                   for idx, (variable, result) in enumerate(results.items(), start=1):
                       # Create a plot of residuals
                       plt.figure(figsize=(6, 4))
                       plt.scatter(range(len(result.resid)), result.resid, alpha=0.7)
                       plt.axhline(0, color="red", linestyle="--", linewidth=1)
                       plt.title(f"Residuals for {variable}")
                       plt.xlabel("Observation")
                       plt.ylabel("Residuals")
                       plt.tight_layout()

                       # Save the plot to a BytesIO object
                       img_data = BytesIO()
                       plt.savefig(img_data, format="png")
                       plt.close()
                       img_data.seek(0)

                       # Add the image to the Excel sheet
                       img = Image(img_data)
                       img.anchor = f"A{idx * 20}"  # Adjust the position of the image
                       residuals_sheet.add_image(img)

                   for sheet_name in ["Coefficients", "Statistics", "Metadata"]:
                       if sheet_name in workbook.sheetnames:
                           sheet = workbook[sheet_name]

                           # Enable text wrapping and adjust row heights
                           for row in sheet.iter_rows():
                               for cell in row:
                                   cell.alignment = Alignment(wrap_text=True)  # Enable text wrapping
                               # Adjust row height
                               sheet.row_dimensions[row[0].row].height = 25  # Set row height to 25 (adjust as needed)

                   # Save the formatted workbook
                   workbook.save(output_path)
                   break  # Exit the loop if writing succeeds
               except PermissionError:
                   #logger.warning(f"File '{output_path}' is currently open. Please close the file and press Enter to try again.")
                   input("Press Enter after closing the file...")
                   time.sleep(5)  # Optional: wait before retrying


# Define Group Iterator

In [28]:
def process_groups_with_processor(dataframe, group_col, date_col, processor_class, processor_args, max_iter=100):
    """
    Processes variables in a DataFrame using a specified processor class and arguments.

    Args:
        dataframe (pd.DataFrame): The input DataFrame containing the data.
        group_col (str): The column name used to group the data.
        date_col (str): The column name containing the date information.
        processor_class (class): The processor class to be used (e.g., GEEProcessor).
        processor_args (dict): Arguments to be passed to the processor class.
        max_iter (int): Maximum number of iterations for the processor's fit method.

    Returns:
        dict: A dictionary containing the results for each variable.
    """
    results = {}
    for variable in dataframe[group_col].unique():
        variable_data = dataframe[dataframe[group_col] == variable].sort_values(by=date_col)
        variable_data = variable_data.reset_index(drop=True)
        
        # Instantiate the processor with the provided arguments
        processor = processor_class(data=variable_data, **processor_args)
        
        # Fit the processor
        processor.fit(maxiter=max_iter)
        
        # Store the results
        results[variable] = processor.results
        
        # Print the summary
        print(processor.results.summary())
    
    return results

# EXEC --- Instantiate Processor and Process/Export

In [29]:

processor_args = PROCESSOR_ARGS

variable_processor = VariableProcessor(
    df = RegressionDF,
    processor_args = processor_args,
    processor = RegressionProcessor,#ARIMAXRegressor, #
    evaluate = True,
    test_size=0.2,
    random_state=42,
)


variable_processor.process_variables()
results = variable_processor.results
evaluation_results = variable_processor.evaluation_results
evaluation_results


StopIteration: 

StopIteration: 

In [30]:
from statsmodels.stats.power import TTestPower

# Parameters
effect_size = 0.2  # Standardized (e.g., Cohen's d)
alpha = 0.05        # Significance level
nobs = 107/2          # Sample size

# Calculate power
power = TTestPower().power(effect_size, nobs, alpha)
print(f"Power: {power:.2f}")  # e.g., 0.67 → 67% chance to detect the effect

Power: 0.30


In [31]:
def plot_residuals_vs_fitted(result, variable):
   residuals = result.resid
   fitted_values = result.fittedvalues
   plt.figure(figsize=(8, 6))
   plt.scatter(fitted_values, residuals)
   plt.xlabel("Fitted Values")
   plt.ylabel("Residuals")
   plt.title(f"Residuals vs. Fitted Values ({variable})")
   plt.axhline(y=0, color='r', linestyle='--')  # Add a horizontal line at y=0
   plt.show()

In [32]:

results = variable_processor.results
for variable in results:

    print(f"Results for {variable}:")
    result = results[variable]
    print(result.summary())
    #print(result.summary2())

    nob = 107 #### TODO: change to get dinamically
    lags = min(20, np.round(nob/4))
    residuals = result.resid
    #plot_acf(residuals, lags=lags, alpha=0.05)
    #plot_pacf(residuals, lags=lags, alpha=0.05, method="ols")  # OLS-based PACF

    #plot_residuals_vs_fitted(result, variable)



    #print("AIC", result.aic,)  # Akaike Information Criterion
    #print("BIC", result.bic,)  # Bayesian Information Criterion

    print("\n")

Results for sul:
                           GLSAR Regression Results                           
Dep. Variable:                valores   R-squared:                       0.413
Model:                          GLSAR   Adj. R-squared:                  0.372
Method:                 Least Squares   F-statistic:                     15.18
Date:                Sun, 27 Apr 2025   Prob (F-statistic):           2.41e-13
Time:                        13:35:55   Log-Likelihood:                -181.33
No. Observations:                 107   AIC:                             378.7
Df Residuals:                      99   BIC:                             400.0
Df Model:                           7                                         
Covariance Type:                  HAC                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            

# Export Results

In [33]:
export_results(results,
    output_path = LOCAL_OUTPUT_PATH,
    processor_args = PROCESSOR_ARGS,
    preprocessor_args=PREPROCESSOR_ARGS,
    output_gsheet=output_wksheet,
)

# Evaluation

In [34]:
'''
baseline_df = RegressionDF[
    (RegressionDF['PosPandemia_Step'] == 0)
    & (RegressionDF['Pandemia_Step'] == 0)
    #& ~(RegressionDF['Choque_Inicial'] == 1)
    #&~(RegressionDF['Lockdown_2021'] == 1)
]

base_line_args = PROCESSOR_ARGS
base_line_args['formula'] = BASE_MODEL_FORMULA


from statsmodels.stats.power import TTestPower

if False:
    for variable in baseline_df[GROUP_COL_NAME].unique():
        variable_data = baseline_df[baseline_df[GROUP_COL_NAME] == variable].sort_values(by=DATE_NAME)
        variable_data = variable_data.reset_index(drop=True)
        X = variable_data[INDEPENDENT_VAR]
        y = variable_data[DEPENDENT_VAR]

        X_std = (X - X.mean()) / X.std()
        Y_std = (y - y.mean()) / y.std()

        power_test_processor = VariableProcessor(
        df=variable_data,
        processor_args=base_line_args,
        processor=RegressionProcessor,
        test_size=0.25,
        random_state=42,
        evaluate = False,
    )
        model, _ = power_test_processor.process_variables()

        effect_size = model[variable].params[1]  # Coeficiente de X_std
        print(f"{variable}_Effect Size (Cohen's d): {effect_size:.2f}")


        # Parameters
        #effect_size = 0.2  # Standardized (e.g., Cohen's d)
        alpha = 0.05        # Significance level
        nobs = variable_data.shape[0]         # Sample size

        # Calculate power
        power = TTestPower().power(effect_size, nobs, alpha)
        print(f"Power: {power:.2f}")  # e.g., 0.67 → 67% chance to detect the effect

# Processar baseline

baseline_processor = VariableProcessor(
    df=baseline_df,
    processor_args=base_line_args,
    processor=RegressionProcessor,
    test_size=0.25,
    random_state=42,
    evaluate = True
)
_, base_line_evaluation = baseline_processor.process_variables()
base_line_evaluation
'''


'\nbaseline_df = RegressionDF[\n    (RegressionDF[\'PosPandemia_Step\'] == 0)\n    & (RegressionDF[\'Pandemia_Step\'] == 0)\n    #& ~(RegressionDF[\'Choque_Inicial\'] == 1)\n    #&~(RegressionDF[\'Lockdown_2021\'] == 1)\n]\n\nbase_line_args = PROCESSOR_ARGS\nbase_line_args[\'formula\'] = BASE_MODEL_FORMULA\n\n\nfrom statsmodels.stats.power import TTestPower\n\nif False:\n    for variable in baseline_df[GROUP_COL_NAME].unique():\n        variable_data = baseline_df[baseline_df[GROUP_COL_NAME] == variable].sort_values(by=DATE_NAME)\n        variable_data = variable_data.reset_index(drop=True)\n        X = variable_data[INDEPENDENT_VAR]\n        y = variable_data[DEPENDENT_VAR]\n\n        X_std = (X - X.mean()) / X.std()\n        Y_std = (y - y.mean()) / y.std()\n\n        power_test_processor = VariableProcessor(\n        df=variable_data,\n        processor_args=base_line_args,\n        processor=RegressionProcessor,\n        test_size=0.25,\n        random_state=42,\n        evaluate =

In [35]:
'''
pos_intervention_df = RegressionDF[RegressionDF['Step_PosPandemia'] == 1]
# Processar pós-intervenção
pos_intervention_processor = VariableProcessor(
    df=pos_intervention_df,
    processor_args=PROCESSOR_ARGS,
    processor=RegressionProcessor,
    test_size=0.2,
    random_state=42,
    evaluate = True
)
_, pos_intervention_evaluation = pos_intervention_processor.process_variables()

pos_intervention_evaluation


for variable, row in pos_intervention_evaluation.iterrows():
    print(f"Results for {variable}:")
    splitted_data = row['splitted_data']  # Access the 'splitted_data' column
    train_data = splitted_data['train_data'].sort_values(by=DATE_NAME)
    test_data = splitted_data['test_data'].sort_values(by=DATE_NAME)

    print("Train Data:")
    print(train_data)

    print("Test Data:")
    print(test_data)

pos_intervention_evaluation
'''

'\npos_intervention_df = RegressionDF[RegressionDF[\'Step_PosPandemia\'] == 1]\n# Processar pós-intervenção\npos_intervention_processor = VariableProcessor(\n    df=pos_intervention_df,\n    processor_args=PROCESSOR_ARGS,\n    processor=RegressionProcessor,\n    test_size=0.2,\n    random_state=42,\n    evaluate = True\n)\n_, pos_intervention_evaluation = pos_intervention_processor.process_variables()\n\npos_intervention_evaluation\n\n\nfor variable, row in pos_intervention_evaluation.iterrows():\n    print(f"Results for {variable}:")\n    splitted_data = row[\'splitted_data\']  # Access the \'splitted_data\' column\n    train_data = splitted_data[\'train_data\'].sort_values(by=DATE_NAME)\n    test_data = splitted_data[\'test_data\'].sort_values(by=DATE_NAME)\n\n    print("Train Data:")\n    print(train_data)\n\n    print("Test Data:")\n    print(test_data)\n\npos_intervention_evaluation\n'

# Teste Lags

In [36]:
#teste_lag
from collections import defaultdict
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import statsmodels.stats.api as sms


DO_LAG_TEST = False
if DO_LAG_TEST:
    lags_to_test = [1, 2, 3, 4, 5, 6, 8, 10]
    estatisticas_por_lag = defaultdict(dict)

    # Adicionar uma variável para armazenar a informação de ACF
    acf_values = defaultdict(dict)

    for lag in lags_to_test:



        processor_args["cov_kwargs"] = {"maxlags": lag}

        variable_processor = VariableProcessor(
            df=RegressionDF,
            processor_args=processor_args,
            processor=RegressionProcessor
        )

        variable_processor.process_variables()
        results = variable_processor.results

        for var, result in results.items():
            # extrair estatísticas desejadas
            summary_frame = result.summary2().tables[1]  # tabelinha de coeficientes
            if "Trend_PosPandemia" in summary_frame.index:
                trend_stats = summary_frame.loc["Trend_PosPandemia"]
                estatisticas_por_lag[lag][var] = {
                    "coef": trend_stats["Coef."],
                    "std_err": trend_stats["Std.Err."],
                    "p_value": trend_stats["P>|z|"] if "P>|z|" in trend_stats else trend_stats["P>|t|"],
                }
            else:
                estatisticas_por_lag[lag][var] = {"coef": None, "std_err": None, "p_value": None}

            # Outras estatísticas: Durbin-Watson, AIC, BIC, R², F-Estatístico, p-valor do modelo
            estatisticas_por_lag[lag][var]["durbin_watson"] = sm.stats.stattools.durbin_watson(result.resid)
            estatisticas_por_lag[lag][var]["aic"] = result.aic
            estatisticas_por_lag[lag][var]["bic"] = result.bic
            estatisticas_por_lag[lag][var]["r_squared"] = result.rsquared
            estatisticas_por_lag[lag][var]["f_statistic"] = result.fvalue
            estatisticas_por_lag[lag][var]["p_value_model"] = result.f_pvalue

            # ACF (Autocorrelation Function) dos resíduos
            acf_result = sm.tsa.acf(result.resid, nlags=lag, fft=True)
            acf_values[lag][var] = acf_result

            # Exibir os valores de ACF para verificação
            print(f"Valores da ACF para {var} no lag {lag}: {acf_result}")

            # Ajuste: Registra os lags com autocorrelação positiva maior que 0.1 ou qualquer critério ajustado
            acf_significant_lags = [i for i, acf_val in enumerate(acf_result) if abs(acf_val) > 0.1]  # Limite mais relaxado
            estatisticas_por_lag[lag][var]["acf_significant_lags"] = acf_significant_lags

    # Converte o dicionário em uma lista de registros
    registros = []
    for lag, variaveis in estatisticas_por_lag.items():
        for var, stats in variaveis.items():
            linha = {"lag": lag, "variavel": var}
            linha.update(stats)
            registros.append(linha)

    # Cria o DataFrame
    df_resultados_lags = pd.DataFrame(registros)

    # Ordena para facilitar a visualização
    df_resultados_lags.sort_values(by=["variavel", "lag"], inplace=True)
    #df_resultados_lags.to_csv("df_resultados_lags.csv", sep=';', decimal=',', index=False, encoding='utf-8-sig')
    # Exibe os resultados



# Define Hover Graph Func


In [37]:
def plot_with_all_dummies(
    variable: str,
    variable_df,
    results_obj,
    exclude_coeffs_by_prefix=None,
    plot_individual_effects=False,
    line_events=None, # Dict[str, Tuple[start, end, color, show_line]]
    slope_coef_str=None,
    real_serie_key=DEPENDENT_VAR,
):

    regression_stats = results_obj.params
    variable_df = variable_df.sort_values(by=DATE_NAME).reset_index(drop=True)
    dates = variable_df[DATE_NAME].values
    real_series = variable_df[real_serie_key].values

    slope_name = slope_coef_str if slope_coef_str else INDEPENDENT_VAR
    intercept = regression_stats.get("Intercept", 0)
    slope = regression_stats.get(slope_name, 0)
    t = np.arange(len(dates)) + 1
    baseline = (intercept + slope * t).astype(float)
    full_pred = baseline.copy()

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=dates,
        y=real_series,
        mode='markers',
        name='Real',
        marker=dict(color='black')
    ))

    fig.add_trace(go.Scatter(
        x=dates,
        y=baseline,
        mode='lines',
        name='Baseline',
        line=dict(dash='dot', color='black')
    ))

    for dummy_name, coef in regression_stats.items():
        if dummy_name in {"Intercept", INDEPENDENT_VAR}:
            continue
        if slope_coef_str and dummy_name == slope_coef_str:
            continue

        if exclude_coeffs_by_prefix and any(dummy_name.startswith(prefix) for prefix in exclude_coeffs_by_prefix):
            continue

        if dummy_name in variable_df.columns:
            dummy_ts = variable_df[dummy_name].values
            dummy_effect = coef * dummy_ts
            full_pred += dummy_effect

            if plot_individual_effects:
                fig.add_trace(go.Scatter(
                    x=dates,
                    y=dummy_effect,
                    mode='lines',
                    name=f'Effect: {dummy_name}',
                    line=dict(dash='dash')
                ))

    if line_events:
        for line_dummy_name, (event_start_date, event_end_date, fillcolor, show_line) in line_events.items():
            if fillcolor:
                # Sombra para eventos tipo "pulse"
                fig.add_shape(
                    type="rect",
                    x0=event_start_date,
                    x1=event_end_date,
                    y0=0,
                    y1=1,
                    xref="x",
                    yref="paper",
                    fillcolor=fillcolor,
                    opacity=0.3,
                    layer="below",
                    line_width=0,
                )
        
            if show_line:
                # Linha e anotação para eventos como Pandemia e Pós-Pandemia
                fig.add_shape(
                    type="line",
                    x0=event_start_date,
                    x1=event_start_date,
                    y0=min(real_series),
                    y1=max(real_series),
                    line=dict(color="black", dash="dash")
                )
        
                # Anotação deslocada no eixo x
                fig.add_annotation(
                    x=event_start_date,
                    y=max(real_series),
                    text=f"{line_dummy_name}",
                    showarrow=True,
                    arrowhead=2,
                    ax=30,  # deslocamento lateral
                    ay=-40
                )

    fig.add_trace(go.Scatter(
        x=dates,
        y=full_pred,
        mode='lines',
        name='Full model',
        line=dict(color='purple')
    ))

    fig.update_layout(
        title=variable,
        xaxis_title="Time",
        yaxis_title="Value",
        legend_title="Legend"
    )

    fig.show()


# EXEC --- Plot Graph

In [39]:
for variable in results:
    variable_result = results[variable]
    variable_df = RegressionDF[RegressionDF[GROUP_COL_NAME] == variable]
    plot_with_all_dummies(variable,
                            variable_df,
                            variable_result,
                            exclude_coeffs_by_prefix=[
                                "month_",
                                'quarter_',
                                'sin','cos'
                                ],
                            line_events = {
                                #'Choque_Inicial':        [CHOQUE_INICIAL_START_DATE, CHOQUE_INICIAL_END_DATE, "rgba(0, 0, 0, 0.25)", False],
                                #'Lockdown_2021':         [SECONDLOCKDOWN_START_DATE, SECONDLOCKDOWN_END_DATE, "rgba(0, 0, 0, 0.25)", False],
                                'Pandemia':         [PANDEMIA_START_DATE, PANDEMIA_END_DATE, None, True],
                                'Pos Pandemia':      [POS_PANDEMIA_START_DATE, END_DATE, None, True]
                            },
                            #real_serie_key=IR_VAR,
                            #slope_coef_str='PrePandemia_Trend'



                            )
